# This is Markdown text?

## Prepare python

In [144]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys



driver = webdriver.Chrome('/Users/gopal/opt/anaconda3/bin/chromedriver')

# browser.get('https://www.facebook.com/')
# sleep(5)
# driver.close()

In [145]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import os
import subprocess
import re
import pandas as pd
import numpy as np

In [146]:
data_filepath = "/Users/gopal/Projects/DataScience/india_soilhealth/data"
index_filepath = "/Users/gopal/Projects/DataScience/india_soilhealth/index"
download_filepath = "/Users/gopal/Downloads/NutrientsStatusReportFarmerWise.csv"

In [147]:
# elem = driver.find_element_by_name("q")
# elem.clear()
# elem.send_keys("pycon")

In [148]:
# elem.send_keys(Keys.RETURN)

In [149]:
driver.get("https://soilhealth.dac.gov.in/PublicReports/NutrientsStatusReportFarmerWise")

In [150]:

# for i in range(len(state_dropdown.options)):
#     if i > 0:
#         print(state_dropdown.options[i].get_attribute('innerText'), state_dropdown.options[i].get_attribute('index'))

In [151]:
# Load states from States dropdown
state_dropdown = Select(driver.find_element_by_name("State_Code"))

states = []
states_index = []
for i in range(len(state_dropdown.options)):
    if i > 0:
        states.append(state_dropdown.options[i].get_attribute('innerText'))
        states_index.append(state_dropdown.options[i].get_attribute('index'))

In [152]:
si = 3
state_dropdown.select_by_index(states_index[si])
state_name = re.sub("\s","",states[si])
sleep(2)
# print(states[si]  + " " + states_index[si])

In [153]:
# Load districts from dropdown
district_dropdown = Select(driver.find_element_by_name("District_Code"))

districts = []
districts_index = []
for i in range(len(district_dropdown.options)):
    if i > 0:
        districts.append(district_dropdown.options[i].get_attribute('innerText'))
        districts_index.append(district_dropdown.options[i].get_attribute('index'))

In [164]:
# Save index of all subdistricts within the district
data = list(zip(np.repeat(states[si], len(villages)), 
                districts))

state_districts_df = pd.DataFrame(data, columns = ['State', 'District'])
state_filepath = os.path.join(index_filepath, state_name + "_districts.csv")
state_districts_df.to_csv(state_filepath)

In [157]:
di = 1
district_dropdown.select_by_index(districts_index[di])
district_name = re.sub("\s","",districts[di])

sleep(2)

# Load subdistricts from dropdown
subdist_dropdown = Select(driver.find_element_by_name("sub_district_code"))

subdists = []
subdist_index = []
for i in range(len(subdist_dropdown.options)):
    if i > 0:
        subdists.append(subdist_dropdown.options[i].get_attribute('innerText'))
        subdist_index.append(subdist_dropdown.options[i].get_attribute('index'))

In [165]:
# Save index of all subdistricts within the district
data = list(zip(np.repeat(states[si], len(villages)), 
                np.repeat(districts[di], len(villages)), 
                subdists))

district_subdists_df = pd.DataFrame(data, columns = ['State', 'District', 'Subdistrict'])
district_filepath = os.path.join(index_filepath, state_name + "-" + district_name + "_subdists.csv")
district_subdists_df.to_csv(district_filepath)

In [159]:
subdist_dropdown.select_by_index(subdist_index[bi])
subdist_name = re.sub("\s","",subdists[bi])

sleep(2)

# Load villages from dropdown
village_dropdown = Select(driver.find_element_by_name("Village_Code"))

villages = []
villages_index = []
for i in range(len(village_dropdown.options)):
    if i > 0:
        villages.append(village_dropdown.options[i].get_attribute('innerText'))
        villages_index.append(village_dropdown.options[i].get_attribute('index'))

In [166]:
# Save index of all villages within the subdistrict
data = list(zip(np.repeat(states[si], len(villages)), 
                np.repeat(districts[di], len(villages)), 
                np.repeat(subdists[bi], len(villages)),
                villages))

subdist_villages_df = pd.DataFrame(data, columns = ['State', 'District', 'Subdistrict', 'Village'])
subdistrict_filepath = os.path.join(index_filepath, state_name + "-" + district_name + "-" + subdist_name + "_villages.csv")
subdist_villages_df.to_csv(subdistrict_filepath)

In [118]:
# Switch to village
for vi in range(len(villages_index)):
    
    village_name = re.sub("\s","",villages[vi])
    village_filename = os.path.join(state_name + "_" + district_name + "_" + subdist_name + "_" + village_name + ".csv")
    village_filepath = os.path.join(data_filepath, village_filename)
    

    if not os.path.isfile(village_filepath):
        driver.switch_to.default_content()

        village_dropdown.select_by_index(villages_index[vi])
        sleep(2)

        view_button = driver.find_element_by_id('confirmLink')
        view_button.click()

        sleep(5)



        # Switch to data display virtual frame
        driver.switch_to.default_content()
        driver.switch_to.frame(0)

        # Save the data
        save_button = driver.find_element_by_id('ReportViewer1_ctl05_ctl04_ctl00_ButtonImg')
        save_button.click()
        sleep(1)
        csv_button = driver.find_element_by_link_text("CSV (comma delimited)")
        csv_button.click()

        driver.switch_to.default_content()

        # 
        sleep(5)
        os.rename(download_filepath, village_filepath)